# Introduction à Python
L'objectif est de découvrir les principes de bases de Python.
Python est un langage interprété, structurée, orientée objet et il existe une très large documentation car c'est un dès plus populaire langage.

In [ ]:
import pandas as pd
import numpy as np

# Créer un dataframe
colonne1 = range(4)
colonne2 = ["a"]*2+["b"]*2

df = pd.DataFrame({"col1" : colonne1, "col2": colonne2})

print(colonne1, "\n", colonne2, "\n")
print(df)

range(0, 4) 
 ['a', 'a', 'b', 'b'] 

   col1 col2
0     0    a
1     1    a
2     2    b
3     3    b


In [ ]:
# Obtenir la dimension du dataframe


In [ ]:
# Obtenir la structure du dataframe


In [ ]:
# Creation d'une fonction
def verifier_pair_impair(num):
  if (num % 2) == 0:
    text = "est pair"
  else:
    text = "est impair"
  return num, text

In [ ]:
verifier_pair_impair(5)

(5, 'est impair')

### MNIST dataset (sample)

### Loading the MNIST dataset in Keras

In [ ]:
# MNIST comes pre-loaded with Keras
from keras.datasets import mnist

# We load it into four Numpy arrays
# train_labels[i] is the label of train_images[i]
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


Let’s take a look at the training data.

In [ ]:
print("Shape of training images:", train_images.shape)
print("Length of training labels:", len(train_labels))
print("Sample of training labels:", train_labels)
print("Same using Numpy repr:", repr(train_labels))

Shape of training images: (60000, 28, 28)
Length of training labels: 60000
Sample of training labels: [5 0 4 ... 5 6 8]
Same using Numpy repr: array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)


Similar look at the test data:

In [ ]:
print("Shape of test images:", test_images.shape)
print("Length of test labels:", len(test_labels))
print("Sample of test labels:", repr(test_labels))

Shape of test images: (10000, 28, 28)
Length of test labels: 10000
Sample of test labels: array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)


## Basic workflow

### The network architecture

The code below builds our network. Don't worry of you don't understand everything about this example yet.

### Preparing for training: the compilation step

Loss functions and optimizers will be discussed more later on.

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

### Preparing the image data

I have modified the book code a bit so the preprocessed data is stored separate arrays (instead of overwriting the original ones). In my opinion it is usually a good idea to keep the original data untouched --- unless memory becomes an issue.

In [ ]:
train_data = train_images.reshape((60000, 28 * 28))
train_data = train_data.astype('float32') / 255
test_data = test_images.reshape((10000, 28 * 28))
test_data = test_data.astype('float32') / 255
# some sanity checks
print(train_data.shape, test_data.shape)

(60000, 784) (10000, 784)


### Preparing the labels

Keras routine `to_categorical` takes care of encoding our labels to so called one-hot -vectors. 

In [ ]:
from tensorflow.keras.utils import to_categorical

train_labels_enc = to_categorical(train_labels)
test_labels_enc = to_categorical(test_labels)
# sanity checks
print(train_labels.shape, train_labels_enc.shape)
print(train_labels[0], 'is encoded as ', train_labels_enc[0])

(60000,) (60000, 10)
5 is encoded as  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


### Training the network

`batch_size`defines the size of data lumps and `epochs`the number of iterations.

Note the quantities displayed during training:
- epoch counter / number of epochs
- data counter / size of data
- training time per epoch
- average training time per "data lump"
- average loss (over one epoch)
- average accuracy (ditto)

Call returns a `History`object consisting of our measurements (defined at earlier `compile` call).

In [ ]:
history = network.fit(train_data, train_labels_enc, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.2531 - accuracy: 0.9274
Epoch 2/5
469/469 [==============================] - 6s 12ms/step - loss: 0.1002 - accuracy: 0.9705
Epoch 3/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0675 - accuracy: 0.9797
Epoch 4/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0491 - accuracy: 0.9852
Epoch 5/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0368 - accuracy: 0.9884


In [ ]:
print(history.history)

{'loss': [0.25307697057724, 0.10016775131225586, 0.06752956658601761, 0.04908774420619011, 0.03677215799689293], 'accuracy': [0.9273666739463806, 0.9704833626747131, 0.9796833395957947, 0.9852499961853027, 0.9884166717529297]}


### Testing the network

A call to `evaluate`returns the metrics we defined when compiling the model (averaged over the test set).

In [ ]:
test_loss, test_acc = network.evaluate(test_data, test_labels_enc)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0789 - accuracy: 0.9762
Test accuracy: 0.9761999845504761


### Some testing on preprocessing 

In [ ]:
# stupid test: do the same without scaling the data

train_data = train_images.reshape((60000, 28 * 28))
train_data = train_data.astype('float32')
test_data = test_images.reshape((10000, 28 * 28))
test_data = test_data.astype('float32')
# some sanity checks
print(train_data.shape, test_data.shape)

# rebuild network (we want to start training from scratch)

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

(60000, 784) (10000, 784)


In [ ]:
# We here train for 20 epochs with bad values at start and not much progress
history = network.fit(train_data, train_labels_enc, epochs=20, batch_size=128)

Epoch 1/20
469/469 [==============================] - 5s 11ms/step - loss: 4.8474 - accuracy: 0.9002
Epoch 2/20
469/469 [==============================] - 5s 10ms/step - loss: 0.6519 - accuracy: 0.9523
Epoch 3/20
469/469 [==============================] - 5s 10ms/step - loss: 0.4351 - accuracy: 0.9644
Epoch 4/20
469/469 [==============================] - 5s 10ms/step - loss: 0.3678 - accuracy: 0.9727
Epoch 5/20
469/469 [==============================] - 5s 10ms/step - loss: 0.3212 - accuracy: 0.9772
Epoch 6/20
469/469 [==============================] - 5s 10ms/step - loss: 0.2885 - accuracy: 0.9792
Epoch 7/20
469/469 [==============================] - 5s 10ms/step - loss: 0.2408 - accuracy: 0.9831
Epoch 8/20
469/469 [==============================] - 5s 10ms/step - loss: 0.2459 - accuracy: 0.9834
Epoch 9/20
469/469 [==============================] - 5s 10ms/step - loss: 0.2174 - accuracy: 0.9857
Epoch 10/20
469/469 [==============================] - 5s 10ms/step - loss: 0.2028 - accura

In [ ]:
test_loss, test_acc = network.evaluate(test_data, test_labels_enc)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.9297 - accuracy: 0.9756
Test accuracy: 0.975600004196167


In [ ]:
# less stupid test: scale input to [-1, 1]

train_data = train_images.reshape((60000, 28 * 28))
train_data = train_data.astype('float32') / 255
train_data = (2 * train_data - 1)

test_data = test_images.reshape((10000, 28 * 28))
test_data = test_data.astype('float32') / 255
test_data = (2 * test_data - 1)

# some sanity checks
print(train_data.shape, test_data.shape)

# rebuild network (we want to start training from scratch)

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

(60000, 784) (10000, 784)


In [ ]:
history = network.fit(train_data, train_labels_enc, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 5s 10ms/step - loss: 0.4188 - accuracy: 0.8787
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1528 - accuracy: 0.9533
Epoch 3/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1094 - accuracy: 0.9660
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0899 - accuracy: 0.9723
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0751 - accuracy: 0.9765


In [ ]:
test_loss, test_acc = network.evaluate(test_data, test_labels_enc)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.0958 - accuracy: 0.9717
Test accuracy: 0.9717000126838684
